# Objective Function
## Weighted sum of Sharpe Ration and ESG Score

The Sharpe Ratio is defined as:

$$
\text{Sharpe Ratio} = \frac{\sum_{i=1}^n w_i \mu_i - R_f}{\sqrt{w^T \Sigma w}}
$$

Where:
- $μ_i$ = Portfolio return  
- $R_f$ = Risk-free rate  
- $σ_p$ = Portfolio volatility (standard deviation)
- $Σw$	= Covariance matrix

In [15]:
def sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate = 0.24, annualize = True):
    weights = np.array(weights)
    expected_returns = np.array(expected_returns)
    cov_matrix = np.array(cov_matrix)
    
    # Portfolio metrics
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    # Annualize if using daily/monthly data
    if not annualize:
        portfolio_return *= 252  
        portfolio_volatility *= np.sqrt(252)
    
    # Sharpe Ratio calculation
    sharpe = (portfolio_return - risk_free_rate) / portfolio_volatility
    
    return sharpe

In [27]:
# initialize weights
# gather ticker info from an API (probably Yahoo Finance)
# calculate expected returns (find best way to do that)
# calculate covariance matrix

# Gather ESG scores

# combine objective functions with weights


# Initialize ABC algorithm to adjust weights (sum of 1) of each stock in order to find the best allocation of stocks in the portfolio
# Backtest on data and confirm it outperforms certain benchmarks
